Assumptions and initial data are expressed below

In [25]:
import math

# Constants
SpecificHeatCapacityOfWater = 4184  # J/kg·K
LatentHeatOfFusion = 334000         # J/kg
SpecificHeatCapacityOfIce = 2090    # J/kg·K
DensityOfWater = 1000               # kg/m³

def CalculateTimeToCool(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity):
    # Derived properties
    r_inner = ((3 * InnerVolume) / (4 * math.pi)) ** (1/3)
    r_outer = r_inner + WallThickness
    WaterMass = InnerVolume * DensityOfWater

    # Energy required in each phase
    EnergyToCoolWater = WaterMass * SpecificHeatCapacityOfWater * max(0, WaterInitTemp - 0)
    EnergyToFreeze = WaterMass * LatentHeatOfFusion
    EnergyToCoolIce = WaterMass * SpecificHeatCapacityOfIce * max(0, 0 - WaterFinalTemp)
    TotalEnergyToRemove = EnergyToCoolWater + EnergyToFreeze + EnergyToCoolIce

    # Time stepping
    time = 0  # seconds
    time_step = 60  # seconds
    energy_removed = 0
    current_temp = WaterInitTemp
    while WaterFinalTemp*0.99 < current_temp:  # Stop when close to final temperature
        # Estimate current internal temperature
        if energy_removed < EnergyToCoolWater:
            current_temp = WaterInitTemp - (energy_removed / EnergyToCoolWater) * (WaterInitTemp - 0)
        elif energy_removed < (EnergyToCoolWater + EnergyToFreeze):
            current_temp = 0  # Freezing stage
        else:
            frac = (energy_removed - EnergyToCoolWater - EnergyToFreeze) / EnergyToCoolIce
            current_temp = 0 + frac * (WaterFinalTemp - 0)

        # Accurate radial conduction for a spherical shell
        Q = 4 * math.pi * HeatConductivity * (r_inner * r_outer) / (r_outer - r_inner) * (current_temp - AmbientTemp)

        energy_this_step = Q * time_step
        energy_removed += energy_this_step
        # Update time
        time += time_step

    return time / 3600  # Convert to hours

# Example usage
WaterInitTemp = 20  # °C
WaterFinalTemp = -80  # °C
AmbientTemp = -80  # °C
InnerVolume = 0.0041  # m³ (4.1 liters)
WallThickness = 0.07  # m
HeatConductivity = 0.033  # W/m·K (styrofoam)

hours = CalculateTimeToCool(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity)
print(f"Time (hours): {hours:.2f} to cool from : {WaterInitTemp}°C to {WaterFinalTemp}°C")

def CalculateTimeToWarm(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity):
    # Derived properties
    r_inner = ((3 * InnerVolume) / (4 * math.pi)) ** (1/3)
    r_outer = r_inner + WallThickness
    WaterMass = InnerVolume * DensityOfWater

    # Energy required to warm the ice from WaterInitTemp to WaterFinalTemp
    EnergyToWarm = WaterMass * SpecificHeatCapacityOfIce * (WaterFinalTemp - WaterInitTemp)

    # Time stepping
    time = 0  # seconds
    time_step = 60  # seconds
    energy_added = 0
    current_temp = WaterInitTemp

    while current_temp < WaterFinalTemp * 1.01:  # Stop when close to final temperature
        # Estimate current temperature based on energy added
        frac = energy_added / EnergyToWarm
        current_temp = WaterInitTemp + frac * (WaterFinalTemp - WaterInitTemp)

        # Heat transfer rate into the sphere
        Q = 4 * math.pi * HeatConductivity * (r_inner * r_outer) / (r_outer - r_inner) * (AmbientTemp - current_temp)

        energy_this_step = Q * time_step
        energy_added += energy_this_step

        # Update time
        time += time_step

    return time / 3600  # Convert to hours

# Example usage
WaterInitTemp = -30  # °C
WaterFinalTemp = -10  # °C
AmbientTemp = -10  # °C (freezer)
InnerVolume = 0.0041  # m³ (4.1 liters)
WallThickness = 0.07  # m
HeatConductivity = 0.033  # W/m·K (styrofoam)

hours = CalculateTimeToWarm(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity)
print(f"Time (hours): {hours:.2f} to warm from : {WaterInitTemp}°C to {WaterFinalTemp}°C")


Time (hours): 168.50 to cool from : 20°C to -80°C
Time (hours): 126.63 to warm from : -30°C to -10°C
